In [1]:
import numpy as np
import tensorflow as tf

### データの準備
# データセットの読み込み
train_dataset = np.genfromtxt("./class_train.csv", delimiter=',', dtype=[float, float, float, "S32"])
test_dataset = np.genfromtxt("./class_test.csv", delimiter=',', dtype=[float, float, float, "S32"])

#データをシャッフル
np.random.shuffle(train_dataset)
np.random.shuffle(test_dataset)

In [2]:
def get_labels(dataset):
    """ラベル(正解データ)を1ofKベクトルに変換する"""
    raw_labels = [item[3] for item in dataset]
    labels = []
    for l in raw_labels:
        if l == "classA":
            labels.append([1.0,0.0,0.0])
        elif l == "classB":
            labels.append([0.0,1.0,0.0])
        elif l == "classC":
            labels.append([0.0,0.0,1.0])
    return np.array(labels)

def get_data(dataset):
    """データセットをnparrayに変換する"""
    raw_data = [list(item)[:3] for item in dataset]
    return np.array(raw_data)

In [3]:
# ラベル
t_train = get_labels(train_dataset)
# データ
x_train = get_data(train_dataset)

In [5]:
t_test = get_labels(test_dataset)
x_test = get_data(test_dataset)

In [6]:
# ラベルを格納するPlaceholder
t = tf.placeholder(tf.float32, shape=(None,3))
# データを格納するPlaceholder
X = tf.placeholder(tf.float32, shape=(None,3))

def single_layer(X):
    """隠れ層"""
    node_num = 30
    w = tf.Variable(tf.truncated_normal([3,node_num]))
    b = tf.Variable(tf.zeros([node_num]))
    f = tf.matmul(X, w) + b
    layer = tf.nn.relu(f)
    return layer

def output_layer(layer):
    """出力層"""
    node_num = 30
    w = tf.Variable(tf.zeros([node_num,3]))
    b = tf.Variable(tf.zeros([3]))
    f = tf.matmul(layer, w) + b
    p = tf.nn.softmax(f)
    return p

# 隠れ層
hidden_layer = single_layer(X)
# 出力層
p = output_layer(hidden_layer)

# 交差エントロピー
cross_entropy = t * tf.log(p)
# 誤差関数
loss = -tf.reduce_mean(cross_entropy)
# トレーニングアルゴリズム
# 勾配降下法 学習率0.1
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
train_step = optimizer.minimize(loss)
# モデルの予測と正解が一致しているか調べる
correct_pred = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
# モデルの精度
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [7]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
i = 0
for _ in range(2000):
    i += 1
    # トレーニング
    sess.run(train_step, feed_dict={X:x_train,t:t_train})
    # 200ステップごとに精度を出力
    if i % 200 == 0:
        # コストと精度を出力
        train_loss, train_acc = sess.run([loss, accuracy], feed_dict={X:x_train,t:t_train})
        # テスト用データを使って評価
        test_loss, test_acc = sess.run([loss, accuracy], feed_dict={X:x_test,t:t_test})
        print "Step: %d" % i
        print "[Train] cost: %f, acc: %f" % (train_loss, train_acc)
        print "[Test] cost: %f, acc: %f" % (test_loss, test_acc)

Step: 200
[Train] cost: 0.179964, acc: 0.783333
[Test] cost: 0.193735, acc: 0.744444
Step: 400
[Train] cost: 0.133637, acc: 0.910000
[Test] cost: 0.148840, acc: 0.855556
Step: 600
[Train] cost: 0.099130, acc: 0.973333
[Test] cost: 0.109639, acc: 0.933333
Step: 800
[Train] cost: 0.072292, acc: 0.990000
[Test] cost: 0.079158, acc: 0.977778
Step: 1000
[Train] cost: 0.049095, acc: 0.990000
[Test] cost: 0.054772, acc: 1.000000
Step: 1200
[Train] cost: 0.035220, acc: 1.000000
[Test] cost: 0.040222, acc: 1.000000
Step: 1400
[Train] cost: 0.026494, acc: 1.000000
[Test] cost: 0.030829, acc: 1.000000
Step: 1600
[Train] cost: 0.020881, acc: 1.000000
[Test] cost: 0.024762, acc: 1.000000
Step: 1800
[Train] cost: 0.017069, acc: 1.000000
[Test] cost: 0.020570, acc: 1.000000
Step: 2000
[Train] cost: 0.014351, acc: 1.000000
[Test] cost: 0.017538, acc: 1.000000


In [9]:
test = np.array([8.3,8.1,8.2])
test = np.array([test])
ans = sess.run(p,feed_dict={X:test})
print(ans)
tmp = np.argmax(ans,axis=1)
if(tmp == 0):
  print("classA")
elif(tmp == 1):
  print("classB")
elif(tmp == 2):
  print("classC")

test = np.array([0.23,2.11,1.15])
test = np.array([test])
ans = sess.run(p,feed_dict={X:test})
print(ans)
tmp = np.argmax(ans,axis=1)
if(tmp == 0):
  print("classA")
elif(tmp == 1):
  print("classB")
elif(tmp == 2):
  print("classC")

[[  9.92181301e-01   7.81868119e-03   1.23376628e-10]]
classA
[[  7.11938029e-08   2.38032592e-03   9.97619569e-01]]
classC
